# ripple texture demo

*<<< check out other demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>*
  
press ctrl+F9 to run all cells in this notebook, or press shift+enter to run each cell sequentially

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*
### be patient :)

the next code cell may take a while because running it causes several things to happen:
- connect to a google colab server -> download the fullcontrol code -> install the fullcontrol code

check out [other tutorials](https://github.com/FullControlXYZ/fullcontrol/blob/master/tutorials/README.md) to understand the python code for the FullControl design

In [7]:
if 'google.colab' in str(get_ipython()):
  !pip install git+https://github.com/ChrisKeeleyGitHub/fullcontrol --quiet
  !pip install trimesh --quiet
import fullcontrol as fc
from google.colab import files
from math import cos, tau, sin, radians
from copy import deepcopy
import lab.fullcontrol as fclab

try:
    import trimesh  # type: ignore
except ModuleNotFoundError:
    trimesh = None

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# base STL helper utilities derived from trimesh_profile_slicing.ipynb
import numpy as np
from math import pi, sin, cos


def ensure_trimesh_loaded():
    global trimesh
    if 'trimesh' not in globals() or trimesh is None:
        import importlib
        trimesh = importlib.import_module('trimesh')
    return trimesh


def upload_file_and_get_name():
    uploaded = files.upload()
    return list(uploaded.keys())[0]


def center_mesh(mesh):
    t = -mesh.centroid
    t[2] = 0
    mesh.apply_translation(t)
    return mesh


def rotate_to_bottom(profile):
    if not profile:
        return profile
    arr = np.array(profile)
    min_index = int(np.argmin(arr[:, 1]))
    return profile[min_index:] + profile[:min_index]


def filter_top_bottom(profile, tol_ratio=0.01):
    if not profile:
        return profile
    arr = np.array(profile)
    z_vals = arr[:, 1]
    z_min, z_max = np.min(z_vals), np.max(z_vals)
    if z_max - z_min == 0:
        return profile
    tol = tol_ratio * (z_max - z_min)
    bottom_idxs = [i for i, z in enumerate(z_vals) if abs(z - z_min) < tol]
    top_idxs = [i for i, z in enumerate(z_vals) if abs(z - z_max) < tol]

    def collapse_cluster(idxs):
        if not idxs:
            return None
        sub = arr[idxs]
        return idxs[int(np.argmax(sub[:, 0]))]

    bottom_keep = collapse_cluster(bottom_idxs)
    top_keep = collapse_cluster(top_idxs)
    new_profile = []
    for i, pt in enumerate(profile):
        if abs(pt[1] - z_min) < tol:
            if i == bottom_keep:
                new_profile.append(pt)
        elif abs(pt[1] - z_max) < tol:
            if i == top_keep:
                new_profile.append(pt)
        else:
            new_profile.append(pt)
    return new_profile


def ensure_anticlockwise(profile):
    if len(profile) < 3:
        return profile
    area = 0.0
    for i in range(len(profile)):
        x1, y1 = profile[i]
        x2, y2 = profile[(i + 1) % len(profile)]
        area += (x1 * y2 - x2 * y1)
    if area < 0:
        return profile[::-1]
    return profile


def get_ordered_profile(sec, a, tol_ratio=0.01):
    if sec is None:
        return []
    if sec.entities is not None and len(sec.entities) > 0:
        try:
            inds = sec.entities[0].points
            pts = sec.vertices[inds]
        except Exception:
            pts = sec.vertices
    else:
        pts = sec.vertices
    U = np.array([cos(a), sin(a), 0])
    local_r = pts.dot(U)
    local_z = pts[:, 2]
    mask = local_r > 0
    pts_local = np.column_stack((local_r[mask], local_z[mask]))
    profile = list(map(tuple, pts_local))
    profile = ensure_anticlockwise(profile)
    profile = rotate_to_bottom(profile)
    profile = filter_top_bottom(profile, tol_ratio)
    return profile


def process_mesh(file_path, n_slices=360, tol_ratio=0.01):
    tm = ensure_trimesh_loaded()
    mesh = tm.load_mesh(file_path)
    center_mesh(mesh)
    angles = np.linspace(0, 2 * pi, n_slices, endpoint=False)
    profiles = []
    for a in angles:
        sec = mesh.section(plane_origin=mesh.centroid, plane_normal=[-np.sin(a), np.cos(a), 0])
        prof = get_ordered_profile(sec, a, tol_ratio)
        if prof:
            profiles.append((a, prof))
    return sorted(profiles, key=lambda tup: tup[0])


def build_mesh_lookup(raw_profiles):
    if not raw_profiles:
        return None
    profile_arrays = []
    theta_values = []
    z_min = float('inf')
    z_max = float('-inf')
    for theta, prof in raw_profiles:
        arr = np.array(prof, dtype=float)
        if arr.size == 0 or arr.shape[1] != 2:
            continue
        arr = arr[np.argsort(arr[:, 1])]
        theta_values.append(theta)
        profile_arrays.append(arr)
        z_min = min(z_min, float(arr[:, 1].min()))
        z_max = max(z_max, float(arr[:, 1].max()))
    if not profile_arrays:
        return None
    return {
        'theta_values': np.array(theta_values, dtype=float),
        'profile_arrays': profile_arrays,
        'z_min': z_min,
        'z_max': z_max,
        'height': float(z_max - z_min),
    }


def mesh_radius_at(theta, z_world, lookup, rotation_offset_rad=0.0):
    if lookup is None or not lookup['profile_arrays']:
        return None
    n = len(lookup['profile_arrays'])
    if n == 0:
        return None
    two_pi = 2 * pi
    theta_norm = (theta + rotation_offset_rad) % two_pi
    idx_float = (theta_norm / two_pi) * n
    i0 = int(np.floor(idx_float)) % n
    i1 = (i0 + 1) % n
    frac = float(idx_float - np.floor(idx_float))
    arr0 = lookup['profile_arrays'][i0]
    arr1 = lookup['profile_arrays'][i1]
    z0 = arr0[:, 1]
    r0 = arr0[:, 0]
    z1 = arr1[:, 1]
    r1 = arr1[:, 0]
    z_sample0 = float(np.clip(z_world, z0.min(), z0.max()))
    z_sample1 = float(np.clip(z_world, z1.min(), z1.max()))
    r_interp0 = float(np.interp(z_sample0, z0, r0))
    r_interp1 = float(np.interp(z_sample1, z1, r1))
    return (1.0 - frac) * r_interp0 + frac * r_interp1


def describe_lookup(lookup):
    if lookup is None:
        return 'no mesh data'
    return f"slices={len(lookup['profile_arrays'])}, z-range=({lookup['z_min']:.2f}, {lookup['z_max']:.2f}) mm"

In [90]:
# printer/gcode parameters

design_name = 'ripples'
nozzle_temp = 200
bed_temp = 0
print_speed = 500
fan_percent = 100
printer_name='prusa_i3' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [101]:
# design parameters

inner_rad= 40
# Inner Radius / Offset (mm) - Without an STL this is the nominal inner radius for the ripple shape.
# When use_base_stl is True this value becomes an additional radial offset applied on top of the STL-derived radius before ripple modulation.
# default value: 15 ; guideline range: 10 to 30

height = 40
# Height (mm) - Height of the part
# default value: 40 ; guideline range: 20 to 80

skew_percent = 10
# Twist (%) - How much does the structure twist over its height? 100% means one full rotation anti-clockwise
# default value: 10 ; guideline range: -100 to 100

star_tips = 0
# Star Tips - The number of outward protrusions from a nominally circular geometry - to make a star-like shape
# default value: 4 ; guideline range: 0 to 10

tip_length = 10
# Star Tip Length (mm) - How much does each 'star tip' protrude beyond the inner radius?
# default value: 5 ; guideline range: -20 to 20

bulge = 10
# Bulge (mm) - The geometry bulges out by this amount half way up the structure
# default value: 2 ; guideline range: -20 to 20

nozzle_dia = 0.8
# Nozzle Diameter (mm) - This is used to set a reasonable value for layer height and extrusion rate
# default value: 0.4 ; guideline range: 0.3 to 1.2

ripples_per_layer = 20
# Ripples Per Layer - Number of in-out waves the nozzle performs for each layer. There is actually an extra half-ripple for each layer so that the ripples are offset for each alternating layer
# default value: 50 ; guideline range: 20 to 100

rip_depth = 1.6
# Ripple Depth (mm) - How far the nozzle moves in and out radially for each 'ripple'
# default value: 1 ; guideline range: 0 to 5

shape_factor = 1.25
# Start Tip Pointiness - This affects how pointy the 'star tips' are and can achieve very interesting geometries
# default value: 1.5 ; guideline range: 0.25 to 5

vert_rip_amp = 0.8

vert_ripples_per_layer = 20
# Vertical Ripples Per Layer - Number of vertical waves per layer
# default value: 1 ; guideline range: 0 to 10

use_sine_squared = True
# Use Sine Squared - If True, vertical ripple waves use a sine-squared shape
# default value: False
# Vertical Ripple Amplitude (mm) - wave height along z for non-planar layers. Peaks align with horizontal ripples

RippleSegs = 200 # 2 means the ripple is zig-zag. increase this value to create a smooth wave, but watch out since the generation time will increase
first_layer_E_factor = 0.4 # set to be 1 to double extrusion by the end of the layer, 0.4 adds 40%, which seemed good for me
centre_x, centre_y = 50, 50

use_base_stl = False
# Enable this to drive the ripple geometry from an uploaded STL profile.
base_stl_filename = ''  # leave blank to upload a file when running in Colab
# Provide an explicit path when running outside of Colab.
base_slice_count = 180
base_slice_tol_ratio = 0.01
match_height_to_mesh = True
base_height_scale = 1.0
base_radius_scale = 1.0
base_radius_offset = 0.0
base_rotation_offset_deg = 0.0
min_base_radius = 0.0

In [ ]:
# load and analyse the base STL if enabled
mesh_lookup = None
mesh_height_mm = 0.0
mesh_z_min = 0.0
mesh_z_max = 0.0
mesh_profiles_raw = []

if use_base_stl:
    ensure_trimesh_loaded()
    stl_path = base_stl_filename
    if not stl_path:
        if 'google.colab' in str(get_ipython()):
            stl_path = upload_file_and_get_name()
        else:
            raise ValueError('Set base_stl_filename when use_base_stl is True outside Colab.')
    raw_profiles = process_mesh(stl_path, n_slices=base_slice_count, tol_ratio=base_slice_tol_ratio)
    mesh_profiles_raw = [(theta, prof) for theta, prof in raw_profiles if len(prof) > 1]
    mesh_lookup = build_mesh_lookup(mesh_profiles_raw)
    if mesh_lookup is None or mesh_lookup['height'] <= 0:
        raise RuntimeError('Unable to extract usable radial profiles from the STL.')
    mesh_height_mm = mesh_lookup['height']
    mesh_z_min = mesh_lookup['z_min']
    mesh_z_max = mesh_lookup['z_max']
    if match_height_to_mesh:
        height = mesh_height_mm * base_height_scale
    print(f'Base STL loaded: {stl_path}')
    print(f'  {describe_lookup(mesh_lookup)}')
    if match_height_to_mesh:
        print(f'  Design height set to {height:.2f} mm from mesh data.')
else:
    print('Base STL disabled; procedural core will use inner_rad as before.')

In [102]:
# generate the design (make sure you've run the above cells before running this cell)

EW = nozzle_dia*2.5
EH = nozzle_dia*0.6
centre = fc.Point(x=50, y=50, z=-1.5)
centre_now = deepcopy(centre)
layers = int(height/EH)
layer_segs = (ripples_per_layer+0.5)*RippleSegs
total_segs = layer_segs*layers

# offset the whole procedure to a convenient position on the print bed. initial_z dictates the gap between the nozzle and the bed for the first layer, assuming the model was designed with a first layer z-position of 0
initial_z = 0.8*EH + vert_rip_amp -3
model_offset = fc.Vector(x=centre_x, y=centre_y, z=initial_z)

steps = []
points = []
steps.append(fc.Printer(print_speed=print_speed/2)) # halve print speed for the first layer
mesh_rotation_offset_rad = radians(base_rotation_offset_deg)
height_for_mapping = layers * EH if layers > 0 else max(height, 1e-6)
for t in range(int(layers*layer_segs)):
    t_val = t/layer_segs # tval = 0 to layers
    a_now = t_val*tau*(1+(skew_percent/100)/layers)
    a_now -= tau/4 # make the print start from front middle (near primer line)
    z_linear = t_val * EH
    if use_base_stl and mesh_lookup is not None and mesh_height_mm > 0 and height_for_mapping > 0:
        mesh_z = mesh_z_min + (z_linear / height_for_mapping) * mesh_height_mm
        base_radius = mesh_radius_at(a_now, mesh_z, mesh_lookup, rotation_offset_rad=mesh_rotation_offset_rad)
        if base_radius is None:
            base_radius = inner_rad
        base_radius = base_radius * base_radius_scale + base_radius_offset + inner_rad
        if min_base_radius > 0:
            base_radius = max(base_radius, min_base_radius)
    else:
        base_radius = inner_rad
    # the next equation (r_now) looks more complicated than it is. basically radius is base_radius + radial fluctuation due to ripples (1st line) + radial fluctuation due to the star shape (2nd line) + radial fluctuation due to the bulge (3rd line)
    r_now = (
        base_radius
        + rip_depth*(0.5+(0.5*cos((ripples_per_layer+0.5)*(t_val*tau))))**1
        + (tip_length*(0.5-0.5*cos(star_tips*(t_val*tau)))**shape_factor)
        + (bulge*(sin((centre_now.z/height)*(0.5*tau))))
    )
    angle = (vert_ripples_per_layer + 0.5) * (t_val * tau)
    if use_sine_squared:
        vert_wave = cos(angle) ** 2  # sine squared wave
    else:
        vert_wave = cos(angle)
    centre_now.z = t_val * EH + vert_rip_amp * vert_wave
    if t_val < 1: # 1st layer
        steps.append(fc.ExtrusionGeometry(height=EH+EH*t_val*first_layer_E_factor)) # ramp up extrusion during the first layer since vase mode means the nozzle moves away from the buildplate
    if t_val == 1: # other layers
        steps.append(fc.ExtrusionGeometry(height=EH)) # reduce to the correct height as soon as the nozzle passes the start point of the previous layer
        steps.append(fc.Printer(print_speed = print_speed)) # double print speed after the first layer. this is combined with an instantaneous reduction in extrusion height, meaning volumetric flow rate would remain constant for this transition if first_layer_E_factor=1
    points.append(fc.polar_to_point(centre_now, r_now, a_now))
steps.extend(fclab.fill_base_full(points, int(layer_segs), 1, EW))
steps = fc.move(steps, model_offset)
annotation_pts = []
annotation_labels = []

yay! CONVEX function used :) please cite our CONVEX research study: https://www.researchgate.net/publication/346098541


In [ ]:
# preview the design

# fc.transform(steps, 'plot', fc.PlotControls(zoom=0.4, style='line'))
# hover the cursor over the lines in the plot to check xyz positions of the points in the design

# uncomment the next line to create a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
fc.transform(steps, 'plot', fc.PlotControls(zoom=0.4, style='tube', initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
# fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.4,  initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))


In [104]:
# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,

    initialization_data={
        'primer': 'front_lines_then_y',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)
open(f'{design_name}.gcode', 'w').write(gcode)
files.download(f'{design_name}.gcode')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ================================
# Wooj-Style Mesh (Dual Helix Lattice)
# ================================


In [ ]:
# --- NEW: mesh controls (reuse existing globals like EH, EW, print_speed, etc.) ---
R_mm = 90.0
H_mm = 280.0
N_strands = 28
pitch_mm = 22.0
phase_offset_deg = 6

undulate = True
und_amp_mm = 0.30
und_freq_per_rev = 2.0

micro_twist = True
twist_radial_amp_mm = 0.12
twist_freq_high = 9.0

add_rims = True
rim_perims = 1

min_gap_target_mm = 0.55

use_frustum = False
Rt_mm = 90.0


In [ ]:
# --- NEW: cylindrical mapping using existing math helpers ---
import math

def cyl_xy(R, theta):
    return R*math.cos(theta), R*math.sin(theta)

def theta_for_z(z, pitch):
    return (2*math.pi) * (z / pitch)

def apply_z_undulation(z, theta, enabled, A, f):
    if not enabled or A <= 0:
        return z
    return z + A * math.sin(f * theta)

def apply_radial_twist(R, theta, enabled, A, f):
    if not enabled or A <= 0:
        return R
    return R + A * math.sin(f * theta)

def radius_at_z(z):
    if not use_frustum:
        return R_mm
    return R_mm + (Rt_mm - R_mm) * (z / H_mm)


In [ ]:
# --- NEW: generate continuous helical strands on a cylinder ---
def generate_helix_family(R, H, N, pitch, sign=+1, phase_offset_rad=0.0):
    strands = []
    for i in range(N):
        phi0 = phase_offset_rad + (2*math.pi * i / N)
        pts = []
        z = 0.0
        dz = max(EH * 0.75, 0.18)
        while z <= H:
            theta = sign * theta_for_z(z, pitch) + phi0
            R_base = radius_at_z(z)
            R_eff = apply_radial_twist(R_base, theta, micro_twist, twist_radial_amp_mm, twist_freq_high)
            x, y = cyl_xy(R_eff, theta)
            x += centre_x
            y += centre_y
            z_eff = apply_z_undulation(z, theta, undulate, und_amp_mm, und_freq_per_rev)
            pts.append((x, y, z_eff))
            z += dz
        strands.append(pts)
    return strands

def generate_dual_helix(R, H, N, pitch, phase_offset_deg):
    A = generate_helix_family(R, H, N, pitch, +1, math.radians(+phase_offset_deg/2))
    B = generate_helix_family(R, H, N, pitch, -1, math.radians(-phase_offset_deg/2))
    return A, B


In [ ]:
# --- NEW: spacing check, emitters, and build routine ---
def check_spacing():
    global N_strands, pitch_mm
    Rmid = radius_at_z(H_mm/2)
    ang_spacing = 2*math.pi / N_strands
    lin_spacing = Rmid * ang_spacing - EW
    if lin_spacing < min_gap_target_mm:
        pitch_mm *= 1.05
        print(f'adjust: increased pitch to {pitch_mm:.2f} mm')
        ang_spacing = 2*math.pi / N_strands
        lin_spacing = Rmid * ang_spacing - EW
        if lin_spacing < min_gap_target_mm and N_strands > 2:
            N_strands -= 2
            ang_spacing = 2*math.pi / N_strands
            lin_spacing = Rmid * ang_spacing - EW
            print(f'adjust: reduced strands to {N_strands}')
    print(f'strand spacing @ mid-height: {math.degrees(ang_spacing):.2f} deg, {lin_spacing:.2f} mm')
    crossings_per_100 = (100 / pitch_mm) * N_strands
    print(f'crossings per 100 mm height: {crossings_per_100:.1f}')
    print(f'minimum gap: {lin_spacing:.2f} mm')
    return ang_spacing, lin_spacing

def emit_polyline(strand, steps):
    steps.append(fc.Extruder(on=False))
    steps.append(fc.Point(x=strand[0][0], y=strand[0][1], z=strand[0][2]))
    steps.append(fc.Extruder(on=True))
    for x, y, z in strand[1:]:
        steps.append(fc.Point(x=x, y=y, z=z))
    steps.append(fc.Extruder(on=False))

def emit_rim_perimeters(R, z=0.0, count=1, steps=None):
    if steps is None:
        steps = []
    segs = max(60, int(2*math.pi*R/1.5))
    for c in range(count):
        pts = []
        for i in range(segs+1):
            theta = 2*math.pi * i / segs
            x, y = cyl_xy(R + c*EW, theta)
            pts.append((x + centre_x, y + centre_y, z))
        emit_polyline(pts, steps)
    return steps

def build_wooj_mesh():
    check_spacing()
    A, B = generate_dual_helix(R_mm, H_mm, N_strands, pitch_mm, phase_offset_deg)
    midA = A[0][len(A[0])//2-5:len(A[0])//2+5]
    midB = B[0][len(B[0])//2-5:len(B[0])//2+5]
    print('sample pts A:', midA)
    print('sample pts B:', midB)
    steps = []
    steps.append(fc.Printer(print_speed=print_speed))
    if add_rims:
        emit_rim_perimeters(R_mm, z=0.0, count=rim_perims, steps=steps)
        emit_rim_perimeters(radius_at_z(H_mm), z=H_mm, count=rim_perims, steps=steps)
    for strand in A:
        emit_polyline(strand, steps)
    for strand in B:
        emit_polyline(strand, steps)
    gcode_controls = fc.GcodeControls(
        printer_name=printer_name,
        initialization_data={
            'primer': 'front_lines_then_y',
            'print_speed': print_speed,
            'nozzle_temp': nozzle_temp,
            'bed_temp': bed_temp,
            'fan_percent': fan_percent,
            'extrusion_width': EW,
            'extrusion_height': EH
        })
    gcode = fc.transform(steps, 'gcode', gcode_controls)
    open('mesh_lampshade.gcode', 'w').write(gcode)
    files.download('mesh_lampshade.gcode')


In [ ]:
# --- NEW: run the build ---
build_wooj_mesh()


#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109.
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol